# 03. Hugging Face Pipeline: AI 모델 마법상자
### 과목: NLP Transfer Learning
---
**학습 목표**
1. **Pipeline**이 무엇인지 '마법상자' 비유로 이해한다.
2. 복잡한 코드 없이 번역, 감성분석, 제로샷 분류 등을 **한 줄로** 실행해본다.
3. `sentencepiece` 같은 라이브러리 의존성 문제나 `tf-keras` 충돌 문제를 해결하는 법을 배운다.

### 🍎 쉬운 비유: "AI 마법상자"
- **Pipeline:** 복잡한 내부 기계 장치(모델 로드, 토큰화, 후처리 등)를 몰라도, 상자에 재료(글자)를 넣으면 결과물(번역, 감정)이 뿅 하고 나오는 **마법상자**입니다.
- 우리는 이 상자 안에 어떤 모델(BERT, GPT 등)을 넣을지만 결정하면 됩니다.

### ⚠️ 중요: 라이브러리 호환성 해결 (Kernel Crash 방지)
최근 `transformers`와 `keras` 버전 호환성 문제로 커널이 죽는 경우가 많습니다. 아래 코드를 꼭 먼저 실행해주세요.

In [ ]:
# 1. 필수 라이브러리 설치
# sentencepiece: 번역 모델 등에서 사용하는 텍스트 쪼개기 도구 (없으면 에러 남)
# tf-keras: 최신 Keras 3.x와 Transformers 간의 충돌 방지용
%pip install -q transformers datasets sentencepiece tf-keras kobert-transformers python-dotenv

In [ ]:
import os

# 2. Keras 충돌 방지 설정
# 텐서플로우를 쓰지 않고 파이토치(PyTorch)만 쓸 때 충돌을 막기 위해 설정합니다.
os.environ["TRANSFORMERS_NO_TF"] = "1" 
# Mac 사용자의 경우 MPS(GPU) 가속 활성화 (선택 사항)
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"]="1"

### 1. Pipeline 기본 사용법
`pipeline(태스크 이름, 모델 이름)` 형태로 사용합니다.
모델 이름을 안 쓰면 Hugging Face가 알아서 제일 인기 있는 걸로 골라줍니다.

In [ ]:
from transformers import pipeline
import warnings
warnings.filterwarnings('ignore') # 지저분한 경고 메시지 숨기기

### 2. 기계 번역 (Translation)
한국어를 영어로 번역해주는 모델을 사용해봅시다.
- 모델: `Helsinki-NLP/opus-mt-ko-en` (유명한 다국어 번역 모델)

In [ ]:
# device=-1: CPU 사용 / device=0: GPU 사용 (Mac은 mps 지원 시 사용 가능)
translator = pipeline('translation', model='Helsinki-NLP/opus-mt-ko-en', device=-1)

print("번역기 준비 완료!")

In [ ]:
# 간단한 문장 번역
result = translator('핫도그를 향하여!!!')
print(result)

In [ ]:
# 여러 문장 한꺼번에 번역 (리스트로 넣기)
sentences = [
    '공부를 열심히 하시면 좋겠어요.',
    '수업을 열심히 들으시면 좋겠고요.',
    '실습도 열심히 해주시면 좋겠어요.',
    '코딩 테스트도 까먹지 말고 열심히 합시다.',
    '모두 성공합니다!'
]

results = translator(sentences)
for r in results:
    print(r['translation_text'])

### 3. 감성 분석 (Sentiment Analysis)
문장의 분위기가 긍정적인지 부정적인지 판단합니다.

In [ ]:
# 1. 영어 감성 분석 (기본 모델 사용)
sentiment_clf = pipeline('sentiment-analysis')

print(sentiment_clf("I'm very happy!!!"))
print(sentiment_clf("I hate this food."))

In [ ]:
# 2. 한국어 감성 분석 (NSMC로 학습된 모델 명시)
# sangrimlee 사용자가 올려둔 모델을 가져옵니다.
ko_sentiment_clf = pipeline('sentiment-analysis', model='sangrimlee/bert-base-multilingual-cased-nsmc')

reviews = [
    '어제 배운 Attention이 어려워서 맥주를 한 잔 했어.',
    '공부는 어렵지만 맥주는 시원하더라',
    '하지만 나는 굴하지 않고 열심히 공부할거야!!!'
]

for review in reviews:
    print(f"{review} -> {ko_sentiment_clf(review)}")

### 4. 제로샷 분류 (Zero-shot Classification)
**학습시킨 적 없는 카테고리**로 문장을 분류하는 놀라운 기능입니다.
모델에게 "이 문장이 정치/경제/건강 중 어디에 속해?"라고 물어보면 맥락을 파악해 답을 줍니다.

In [ ]:
# 영어 제로샷 분류
zero_shot_clf = pipeline('zero-shot-classification')

text = "This is a course about the transformers library."
labels = ["education", "politics", "business"]

print(zero_shot_clf(text, candidate_labels=labels))

In [ ]:
# 한국어 제로샷 분류 (다국어 모델 사용)
ko_zero_shot_clf = pipeline('zero-shot-classification', model='joeddav/xlm-roberta-large-xnli')

text = "2025년에는 어떤 운동을 하시겠습니까?!"
labels = ["정치", "경제", "건강"]

# hypothesis_template: 모델에게 힌트를 주는 문장 틀
result = ko_zero_shot_clf(
    text, 
    labels, 
    hypothesis_template="이 텍스트는 {}에 관한 내용입니다."
)
print(result)

### 5. 텍스트 생성 (Text Generation)
GPT처럼 문장의 앞부분을 주면 뒷부분을 이어서 써줍니다.

In [ ]:
# 한국어 GPT 모델 (SKT KoGPT Trinity)
ko_text_generator = pipeline('text-generation', model='skt/ko-gpt-trinity-1.2B-v0.5')

# 문장 이어 쓰기
start_text = "AI란 말입니다."
generated = ko_text_generator(start_text, max_length=50, do_sample=True)

print(generated[0]['generated_text'])

### 6. 빈칸 채우기 (Fill-Mask)
BERT처럼 문장 중간의 구멍(Mask)을 채워주는 기능입니다.

In [ ]:
unmasker = pipeline('fill-mask') # 기본 모델 (DistilRoBERTa)

# <mask> 자리에 들어갈 알맞은 단어 추천
result = unmasker('This course will teach you all about <mask> model.', top_k=3)

for r in result:
    print(f"단어: {r['token_str']}, 점수: {r['score']:.4f}")

### 7. 개체명 인식 (NER)
사람 이름(PER), 장소(LOC), 기관(ORG) 등을 찾아줍니다.

In [ ]:
ner = pipeline('ner', model='dslim/bert-base-NER', grouped_entities=True)

text = "My name is Squirrel and I work at Playdata in Seoul"
print(ner(text))